# Auth

In [25]:
import requests

url = 'https://open.tiktokapis.com/v2/oauth/token/'
headers = {
    'Content-Type': 'application/x-www-form-urlencoded',
    'Cache-Control': 'no-cache'
}
data = {
    'client_key': 'awjr8ug1a6uflwb0',
    'client_secret': 'NXAQZcV4UgsE1b3sEOamy9SzPAa3yD9F',
    'grant_type': 'client_credentials'
}

response = requests.post(url, headers=headers, data=data)

if response.status_code == 200:
    print("Token request successful!")
    token = response.json().get('access_token')
    print(f"Access Token: {token}")
else:
    print(f"Token request failed with status code: {response.status_code}")
    print(response.text)
    token = None

Token request successful!
Access Token: clt.2.gs2D946HdeO9scmxzYk7c9A71UacEzolw3XWnnxsot8FjhXIfGBlAR88xpnS_1JLdKKNm4RNHD3ts9PtI1RDRA*0


# Collect Videos

In [26]:
hashtags = [
    "#LineFire",
    "#LineFireSanBernardino",
    "#LineFireLA",
    "#LineFire2024",
    "#LineFireHighland",
    "#LineFireSouthCalifornia",
    "#LineFireEvacuation",
    "#LineFireEmergency",
    "#LineFireResponse",
    "#LineFireUpdates",
    "#LineFireWildfire",
    "#LineFireSmoke",
    "#LineFireAirQuality",
    "#LineFireDanger",
    "#LineFireFirefighters",
    "#LineFireAlert",
    "#LineFireImpact",
    "#LineFireInTheHighlands",
    "#LineFireOutbreak",
    "#LineFireCoverage",
    "#LineFireSafety",
    "#LineFireLive",
    "#LineFireSpread",
    "#LineFireThreat",
    "#LineFireDestruction"
]

video_keywords = [
    "Line Fire",
    "SanBernardino Fire",
    "Highland Fire",
    "Line Fire Evacuation",
    "Line Fire Response",
    "Line Fire Updates",
    "Line Fire Outbreak",
    "Line Fire Coverage",
    "Line Fire Emergency",
    "Line Fire AirQuality",
    "Line Fire Spread",
    "Line Fire Live"
]

In [27]:
import requests
import json
from time import sleep


url = "https://open.tiktokapis.com/v2/research/video/query/"
params = {
    "fields": "id,video_description,create_time,username,voice_to_text,share_count,view_count,like_count,comment_count,music_id,hashtag_names,is_stem_verified,favorites_count,video_duration"
}
headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}
data = {
    "query": {
        "and": [
            {
                "operation": "IN",
                "field_name": "region_code",
                "field_values": ["US"]
            },
        ],
        "or": [
            {
                "operation": "IN",
                "field_name": "hashtag_name",
                "field_values": hashtags
            },
            {
                "operation": "IN",
                "field_name": "keyword",
                "field_values": video_keywords
            },
        ],
    },
    "max_count": 100,
    "cursor": 0,
    "start_date": "20240906",
    "end_date": "20240921"
}

all_videos = []
has_more = True
cursor = 0
search_id = None

MAX_RETRIES = 100
RETRY_DELAY = 1
max_wanted = 50000

while has_more and len(all_videos) < max_wanted:
    payload = data.copy()
    payload["cursor"] = cursor

    if search_id is not None:
        payload["search_id"] = search_id

    for attempt in range(1, MAX_RETRIES + 1):
        try:
            response = requests.post(url, params=params, headers=headers, json=payload)
        except requests.exceptions.RequestException as e:
            print(f"Request exception on attempt {attempt}: {e}")
            if attempt < MAX_RETRIES:
                print(f"Retrying in {RETRY_DELAY} seconds...")
                sleep(RETRY_DELAY)
                continue
            else:
                has_more = False
                break

        if response.status_code == 200:
            try:
                result = response.json()
            except json.JSONDecodeError:
                print("Failed to parse JSON response.")
                if attempt < MAX_RETRIES:
                    print(f"Retrying in {RETRY_DELAY} seconds...")
                    sleep(RETRY_DELAY)
                    continue
                else:
                    print("Max retries reached due to JSON parsing errors. Exiting.")
                    has_more = False
                    break

            videos = result.get("data", {}).get("videos", [])
            all_videos.extend(videos)
            print(f"Collected: {len(all_videos)} videos")

            data_response = result.get("data", {})
            has_more = data_response.get("has_more", False)
            cursor = data_response.get("cursor")

            if search_id is None:
                search_id = data_response.get("search_id")
                if not search_id:
                    print("No 'search_id' returned by the API. Exiting.")
                    has_more = False
                    break

            break
        else:
            if attempt < MAX_RETRIES:
                sleep(RETRY_DELAY)
                continue
            else:
                print(f"Request failed with status code: {response.status_code}")
                print(response.text)
                print("Max retries reached due to persistent errors. Exiting.")
                has_more = False
                break

    sleep(1)

print(f"Total videos collected: {len(all_videos)}")

final_result = {
    'data': {'videos': all_videos},
    'post_count': len(all_videos)
}

with open('Line_Fire_Tiktok_Video.json', 'w') as f:
    json.dump(final_result, f, indent=2)

print(f"Data has been successfully stored in 'Line_Fire_Tiktok_Video.json'")
print(f"Total number of posts saved: {len(all_videos)}")


Collected: 93 videos
Collected: 184 videos


KeyboardInterrupt: 

# Collect Comments


In [28]:
import requests
import json

comment_url = "https://open.tiktokapis.com/v2/research/video/comment/list/?fields=id,like_count,create_time,text,video_id,parent_comment_id"

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {token}"
}

with open('/content/Line_Fire_Tiktok_Video.json', 'r') as file:
    all_videos = json.load(file)["data"]["videos"]

video_ids = [video['id'] for video in all_videos if video['comment_count'] > 0]

all_comments = []
counter = 1

for video_id in video_ids:
    cursor = 0
    has_more = True
    while has_more:
        data = {
            "video_id": video_id,
            "max_count": 100,
            "cursor": cursor
        }

        response = requests.post(comment_url, headers=headers, json=data)

        if response.status_code == 200:
            result = response.json()
            comments = result.get("data", {}).get("comments", [])
            all_comments.extend(comments)

            has_more = result.get("data", {}).get("has_more", False)
            cursor = result.get("data", {}).get("cursor", None)
            print(f"The {counter} video, Collected {len(comments)} comments")

        else:
            print(f"Request failed for video ID {video_id} with status code: {response.status_code}")
            print(response.text)
            break

    counter += 1

print(f"Totally Collected {len(all_comments)} comments")

with open('all_comments1.json', 'w') as file:
    json.dump(all_comments, file, indent=4)

print("Comments data saved to 'all_comments1.json'")


Request failed for video ID 7413812406570257694 with status code: 429
{"data":{},"error":{"code":"daily_quota_limit_exceeded","message":"The API daily quota limit exceeded. Please try again later","log_id":"20240923043037AE78DC2BFC88AB771303"}}
Request failed for video ID 7413807826901994782 with status code: 429
{"data":{},"error":{"code":"daily_quota_limit_exceeded","message":"The API daily quota limit exceeded. Please try again later","log_id":"20240923043037B9E56041958938694784"}}
Request failed for video ID 7413795485951003947 with status code: 429
{"data":{},"error":{"code":"daily_quota_limit_exceeded","message":"The API daily quota limit exceeded. Please try again later","log_id":"202409230430385FD28D9C1DBDC268F07E"}}
Request failed for video ID 7413794277941153054 with status code: 429
{"data":{},"error":{"code":"daily_quota_limit_exceeded","message":"The API daily quota limit exceeded. Please try again later","log_id":"20240923043038F2261458CC815F694978"}}
Request failed for v

KeyboardInterrupt: 

# Collect User Info

In [ ]:
import requests
import json

url = "https://open.tiktokapis.com/v2/research/user/info/"

params = {
    "fields": "display_name,bio_description,is_verified,follower_count,following_count,likes_count,video_count"
}

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {token}"
}

# Load the previously collected video data
with open("/content/Line_Fire_Tiktok_Video.json", 'r') as file:
    all_videos = json.load(file)["data"]["videos"]

# Extract unique usernames from the collected video data
usernames = list(set(video["username"] for video in all_videos))
print(f"Example username: {usernames[0]}")


user_profiles = []

for username in usernames:
    data = {
        'username': username
    }

    response = requests.post(url, params=params, headers=headers, json=data)

    if response.status_code == 200:
        result = response.json()
        user_profiles.append(result)
    else:
        if response.status_code != 400:
            print(f"Request failed for user: {username}")
            print(f"Status code: {response.status_code}")
            print(response.text)

print(f"Total user profiles retrieved: {len(user_profiles)}")

with open('user_profiles.json', 'w') as file:
    json.dump(user_profiles, file, indent=4)

Example username: earthquakedude
Request failed for user: lion_king4563
Status code: 400
{"error":{"code":"invalid_params","message":"`username: lion_king4563` is invalid: cannot find the user","log_id":"20240923011908BC264A1FA666C361E0D0"}}
Request failed for user: guggu_asr
Status code: 400
{"error":{"code":"invalid_params","message":"`username: guggu_asr` is invalid: cannot find the user","log_id":"202409230119118FE23CC9F36B7D60B9A6"}}
Request failed for user: wbwh38
Status code: 400
{"error":{"code":"invalid_params","message":"`username: wbwh38` is invalid: cannot find the user","log_id":"20240923011957047C15F26103E560AB6D"}}
Request failed for user: go23tti
Status code: 400
{"error":{"code":"invalid_params","message":"API cannot return this user's information","log_id":"20240923011959B11E3860975103619625"}}
Request failed for user: i_wanna_go_home3005
Status code: 400
{"error":{"code":"invalid_params","message":"API cannot return this user's information","log_id":"2024092301200733

In [31]:
import requests
import json

url = "https://open.tiktokapis.com/v2/research/user/liked_videos/"
params = {
    "fields": "id,create_time,username,region_code,video_description,music_id,like_count,comment_count,share_count,view_count,hashtag_names,is_stem_verified,favorites_count,video_duration"
}
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {token}"
}


# Load the previously collected video data
with open("/content/Line_Fire_Tiktok_Video.json", 'r') as file:
    all_videos = json.load(file)["data"]["videos"]

usernames = list(set(video["username"] for video in all_videos))


all_liked_videos = []
for username in usernames:
    cursor = 0
    has_more = True
    user_liked_videos = []

    while has_more:
        data = {
            "username": username,
            "max_count": 100,
            "cursor": cursor
        }
        response = requests.post(url, params=params, headers=headers, json=data)

        if response.status_code == 200:
            result = response.json()
            liked_videos = result.get("data", {}).get("user_liked_videos", [])
            user_liked_videos.extend(liked_videos)
            has_more = result.get("data", {}).get("has_more", False)
            cursor = result.get("data", {}).get("cursor", None)
            print(f"Collected {len(liked_videos)} liked videos for user: {username}")
        else:
            if response.status_code != 403:
                print(f"Request failed for user: {username}")
                print(f"Status code: {response.status_code}")
                print(response.text)
            break

    all_liked_videos.extend(user_liked_videos)
    print(f"Total liked videos collected for {username}: {len(user_liked_videos)}")

print(f"Total liked videos collected across all users: {len(all_liked_videos)}")

# Save all collected liked videos to a JSON file
with open('all_liked_videos.json', 'w') as file:
    json.dump(all_liked_videos, file, indent=4)

print("Liked videos data saved to 'all_liked_videos.json'")

Total liked videos collected for earthquakedude: 0
Total liked videos collected for megan.elizabeth531: 0
Total liked videos collected for empress.cs: 0
Total liked videos collected for alo.miracle: 0
Total liked videos collected for venompool8485: 0
Total liked videos collected for ourdailylife50: 0
Total liked videos collected for sailormoomof44: 0
Total liked videos collected for brittney_malouf92: 0
Total liked videos collected for randii.diana: 0
Total liked videos collected for standard.escrow: 0
Total liked videos collected for elysalc5: 0
Total liked videos collected for zerlenerichard1978: 0
Total liked videos collected for draawlo: 0
Total liked videos collected for cynthiachristine001: 0
Total liked videos collected for kingsfadedi9: 0
Total liked videos collected for emmmmm_____0: 0
Total liked videos collected for naanc16: 0
Total liked videos collected for contessapinklee1: 0
Total liked videos collected for ddfairyy: 0
Total liked videos collected for fitt.ness: 0
Total 

KeyboardInterrupt: 

In [33]:
import requests
import json

url = "https://open.tiktokapis.com/v2/research/user/pinned_videos/"
params = {
    "fields": "id,create_time,username,region_code,video_description,music_id,like_count,comment_count,share_count,view_count,hashtag_names,is_stem_verified,favorites_count,video_duration"
}
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {token}"
}

with open("/content/Line_Fire_Tiktok_Video.json", 'r') as file:
    all_videos = json.load(file)["data"]["videos"]

usernames = list(set(video["username"] for video in all_videos))

all_pinned_videos = []
for username in usernames:
    data = {
        "username": username
    }
    response = requests.post(url, params=params, headers=headers, json=data)

    if response.status_code == 200:
        result = response.json()
        pinned_videos = result.get("data", {}).get("pinned_videos_list", [])
        all_pinned_videos.extend(pinned_videos)
        print(f"Collected {len(pinned_videos)} pinned videos for user: {username}")
    else:
        if response.status_code != 400:
            print(f"Request failed for user: {username}")
            print(f"Status code: {response.status_code}")
            print(response.text)

print(f"Total pinned videos collected across all users: {len(all_pinned_videos)}")

# Save all collected pinned videos to a JSON file
with open('all_pinned_videos.json', 'w') as file:
    json.dump(all_pinned_videos, file, indent=4)

print("Pinned videos data saved to 'all_pinned_videos.json'")

Collected 2 pinned videos for user: earthquakedude
Collected 1 pinned videos for user: megan.elizabeth531
Collected 3 pinned videos for user: empress.cs
Collected 2 pinned videos for user: alo.miracle
Collected 3 pinned videos for user: venompool8485
Collected 3 pinned videos for user: ourdailylife50
Collected 0 pinned videos for user: sailormoomof44
Collected 3 pinned videos for user: brittney_malouf92
Collected 0 pinned videos for user: randii.diana


KeyboardInterrupt: 

In [34]:
import requests
import json

url = "https://open.tiktokapis.com/v2/research/user/reposted_videos/"
params = {
    "fields": "id,create_time,username,region_code,video_description,music_id,like_count,comment_count,share_count,view_count,hashtag_names,is_stem_verified,favorites_count,video_duration"
}
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {token}"
}


with open("/content/Line_Fire_Tiktok_Video.json", 'r') as file:
    all_videos = json.load(file)["data"]["videos"]

usernames = list(set(video["username"] for video in all_videos))

all_reposted_videos = []
for username in usernames:
    cursor = 0
    has_more = True
    user_reposted_videos = []

    while has_more:
        data = {
            "username": username,
            "max_count": 100,
            "cursor": cursor
        }
        response = requests.post(url, params=params, headers=headers, json=data)

        if response.status_code == 200:
            result = response.json()
            reposted_videos = result.get("data", {}).get("reposted_videos", [])
            user_reposted_videos.extend(reposted_videos)
            has_more = result.get("data", {}).get("has_more", False)
            cursor = result.get("data", {}).get("cursor", None)
            print(f"Collected {len(reposted_videos)} reposted videos for user: {username}")
        else:
            print(f"Request failed for user: {username}")
            print(f"Status code: {response.status_code}")
            print(response.text)
            break

    all_reposted_videos.extend(user_reposted_videos)
    print(f"Total reposted videos collected for {username}: {len(user_reposted_videos)}")

print(f"Total reposted videos collected across all users: {len(all_reposted_videos)}")

# Save all collected reposted videos to a JSON file
with open('all_reposted_videos.json', 'w') as file:
    json.dump(all_reposted_videos, file, indent=4)

print("Reposted videos data saved to 'all_reposted_videos.json'")

Collected 54 reposted videos for user: earthquakedude
Total reposted videos collected for earthquakedude: 54
Collected 97 reposted videos for user: megan.elizabeth531
Request failed for user: megan.elizabeth531
Status code: 500
{"error":{"code":"invalid_params","message":"Something is wrong. Please try again later.","log_id":"20240923044617BD2C9CF4A1A5986A8E7E"}}
Total reposted videos collected for megan.elizabeth531: 97
Collected 96 reposted videos for user: empress.cs
Collected 96 reposted videos for user: empress.cs
Collected 95 reposted videos for user: empress.cs
Collected 89 reposted videos for user: empress.cs
Collected 88 reposted videos for user: empress.cs
Collected 86 reposted videos for user: empress.cs
Collected 88 reposted videos for user: empress.cs
Collected 86 reposted videos for user: empress.cs
Collected 81 reposted videos for user: empress.cs


KeyboardInterrupt: 

In [36]:
import requests
import json
import time

url = "https://open.tiktokapis.com/v2/research/user/followers/"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {token}"
}

with open("/content/Line_Fire_Tiktok_Video.json", 'r') as file:
    all_videos = json.load(file)["data"]["videos"]

usernames = list(set(video["username"] for video in all_videos))

all_followers = []
for username in usernames:
    cursor = int(time.time())
    has_more = True
    user_followers = []

    while has_more:
        data = {
            "username": username,
            "max_count": 100,
            "cursor": cursor
        }
        response = requests.post(url, headers=headers, json=data)

        if response.status_code == 200:
            result = response.json()
            followers = result.get("data", {}).get("user_followers", [])
            user_followers.extend(followers)
            has_more = result.get("data", {}).get("has_more", False)
            cursor = result.get("data", {}).get("cursor", None)
            print(f"Collected {len(followers)} followers for user: {username}")
        else:
            if response.status_code != 403:
                print(f"Request failed for user: {username}")
                print(f"Status code: {response.status_code}")
                print(response.text)
            break

    all_followers.append({
        "username": username,
        "followers": user_followers
    })
    print(f"Total followers collected for {username}: {len(user_followers)}")

print(f"Collected followers for {len(all_followers)} users")

# Save all collected followers to a JSON file
with open('all_user_followers.json', 'w') as file:
    json.dump(all_followers, file, indent=4)

print("User followers data saved to 'all_user_followers.json'")

Collected 71 followers for user: earthquakedude
Collected 65 followers for user: earthquakedude
Collected 66 followers for user: earthquakedude
Collected 75 followers for user: earthquakedude
Collected 72 followers for user: earthquakedude
Collected 69 followers for user: earthquakedude
Collected 65 followers for user: earthquakedude
Collected 63 followers for user: earthquakedude
Collected 68 followers for user: earthquakedude
Collected 65 followers for user: earthquakedude
Collected 61 followers for user: earthquakedude
Collected 57 followers for user: earthquakedude
Collected 66 followers for user: earthquakedude


KeyboardInterrupt: 

In [37]:
import requests
import json
import time

url = "https://open.tiktokapis.com/v2/research/user/following/"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {token}"
}

with open("/content/Line_Fire_Tiktok_Video.json", 'r') as file:
    all_videos = json.load(file)["data"]["videos"]

usernames = list(set(video["username"] for video in all_videos))

all_following = []
for username in usernames:
    cursor = int(time.time())
    has_more = True
    user_following = []

    while has_more:
        data = {
            "username": username,
            "max_count": 100,
            "cursor": cursor
        }
        response = requests.post(url, headers=headers, json=data)

        if response.status_code == 200:
            result = response.json()
            following = result.get("data", {}).get("user_following", [])
            user_following.extend(following)
            has_more = result.get("data", {}).get("has_more", False)
            cursor = result.get("data", {}).get("cursor", None)
            print(f"Collected {len(following)} accounts followed by user: {username}")
        else:
            if response.status_code != 403:
                print(f"Request failed for user: {username}")
                print(f"Status code: {response.status_code}")
                print(response.text)
            break

    all_following.append({
        "username": username,
        "following": user_following
    })
    print(f"Total accounts followed by {username}: {len(user_following)}")

print(f"Collected following data for {len(all_following)} users")

# Save all collected following data to a JSON file
with open('all_user_following.json', 'w') as file:
    json.dump(all_following, file, indent=4)

print("User following data saved to 'all_user_following.json'")

Collected 97 accounts followed by user: earthquakedude
Collected 88 accounts followed by user: earthquakedude
Collected 91 accounts followed by user: earthquakedude
Collected 93 accounts followed by user: earthquakedude
Collected 57 accounts followed by user: earthquakedude
Total accounts followed by earthquakedude: 426
Request failed for user: megan.elizabeth531
Status code: 403
{"error":{"code":"forbidden","message":"This information cannot be returned","log_id":"2024092304491818C1CB8CD2128F6ACAB0"}}
Total accounts followed by megan.elizabeth531: 0
Collected 95 accounts followed by user: empress.cs
Collected 92 accounts followed by user: empress.cs
Collected 91 accounts followed by user: empress.cs
Collected 90 accounts followed by user: empress.cs
Collected 89 accounts followed by user: empress.cs
Collected 87 accounts followed by user: empress.cs
Collected 60 accounts followed by user: empress.cs
Total accounts followed by empress.cs: 604
Collected 84 accounts followed by user: alo

KeyboardInterrupt: 